# Agentes de IA para Analisis Geoespacial - Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/08_Agentes_IA_Geoespacial_Pasto.ipynb)

---

## Objetivos

1. Usar agentes de IA para buscar datos geoespaciales
2. Automatizar consultas a catalogos STAC
3. Analizar imagenes con modelos de vision-lenguaje
4. Crear flujos de trabajo automatizados para Pasto

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap pystac-client planetary-computer transformers torch

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import pystac_client
import planetary_computer
import torch
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## 3. Coordenadas de San Juan de Pasto

In [ ]:
# Coordenadas de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}
PASTO_BBOX = (-77.35, 1.15, -77.20, 1.30)

# Areas de interes
AREAS_INTERES = {
    "Centro Urbano": (-77.30, 1.20, -77.26, 1.24),
    "Volcan Galeras": (-77.40, 1.18, -77.32, 1.26),
    "Laguna de la Cocha": (-77.20, 1.08, -77.10, 1.16),
}

print(f"Centro de Pasto: {PASTO_CENTER}")

## 4. Agente de Busqueda en Catalogos STAC

STAC (SpatioTemporal Asset Catalog) es un estandar para catalogar datos geoespaciales.

In [ ]:
class STACSearchAgent:
    """
    Agente para buscar datos en catalogos STAC.
    """

    def __init__(self):
        self.catalog = pystac_client.Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1",
            modifier=planetary_computer.sign_inplace,
        )
        self.colecciones_disponibles = [
            "sentinel-2-l2a",
            "landsat-c2-l2",
            "cop-dem-glo-30",
            "nasadem",
            "io-lulc-annual-v02",
        ]

    def buscar(
        self, bbox, fecha_inicio, fecha_fin, coleccion="sentinel-2-l2a", max_nubes=20
    ):
        """
        Busca imagenes en el catalogo.
        """
        query = (
            {"eo:cloud_cover": {"lt": max_nubes}}
            if "sentinel" in coleccion or "landsat" in coleccion
            else {}
        )

        search = self.catalog.search(
            collections=[coleccion],
            bbox=bbox,
            datetime=f"{fecha_inicio}/{fecha_fin}",
            query=query,
        )

        return list(search.items())

    def describir_colecciones(self):
        """
        Describe las colecciones disponibles.
        """
        descripciones = {
            "sentinel-2-l2a": "Imagenes multiespectrales Sentinel-2 (10-60m)",
            "landsat-c2-l2": "Imagenes Landsat Collection 2 (30m)",
            "cop-dem-glo-30": "Modelo de elevacion Copernicus (30m)",
            "nasadem": "Modelo de elevacion NASA (30m)",
            "io-lulc-annual-v02": "Cobertura del suelo anual (10m)",
        }
        return descripciones


# Crear agente
agente_stac = STACSearchAgent()
print("Colecciones disponibles:")
for col, desc in agente_stac.describir_colecciones().items():
    print(f"  - {col}: {desc}")

In [ ]:
# Buscar imagenes para Pasto
print("Buscando imagenes Sentinel-2 para Pasto...")
resultados = agente_stac.buscar(
    bbox=PASTO_BBOX,
    fecha_inicio="2024-01-01",
    fecha_fin="2024-12-31",
    coleccion="sentinel-2-l2a",
    max_nubes=15,
)

print(f"\nImagenes encontradas: {len(resultados)}")

if resultados:
    print("\nMejores 5 imagenes (por cobertura de nubes):")
    resultados_ordenados = sorted(
        resultados, key=lambda x: x.properties.get("eo:cloud_cover", 100)
    )
    for i, item in enumerate(resultados_ordenados[:5]):
        print(
            f"  {i+1}. {item.datetime.strftime('%Y-%m-%d')} - Nubes: {item.properties.get('eo:cloud_cover'):.1f}%"
        )

## 5. Agente de Analisis de Imagenes

Este agente puede analizar imagenes satelitales y describir su contenido.

In [ ]:
class ImageAnalysisAgent:
    """
    Agente para analizar imagenes satelitales.
    """

    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def analizar_cobertura(self, ndvi, ndwi=None):
        """
        Analiza la cobertura del suelo basado en indices.
        """
        import numpy as np

        total = ndvi.size

        # Clasificacion por NDVI
        agua = (
            np.sum(ndvi < -0.1) / total * 100
            if ndwi is None
            else np.sum(ndwi > 0.1) / total * 100
        )
        urbano = np.sum((ndvi >= -0.1) & (ndvi < 0.15)) / total * 100
        vegetacion_baja = np.sum((ndvi >= 0.15) & (ndvi < 0.3)) / total * 100
        vegetacion_media = np.sum((ndvi >= 0.3) & (ndvi < 0.5)) / total * 100
        vegetacion_alta = np.sum(ndvi >= 0.5) / total * 100

        return {
            "agua_o_nubes": agua,
            "urbano_o_suelo": urbano,
            "vegetacion_baja": vegetacion_baja,
            "vegetacion_media": vegetacion_media,
            "vegetacion_alta": vegetacion_alta,
        }

    def generar_reporte(self, cobertura, area_nombre):
        """
        Genera un reporte textual de la cobertura.
        """
        reporte = f"\n=== Reporte de Cobertura: {area_nombre} ===\n"
        reporte += f"\nDistribucion de cobertura del suelo:\n"

        for tipo, porcentaje in cobertura.items():
            nombre = tipo.replace("_", " ").title()
            barra = "#" * int(porcentaje / 2)
            reporte += f"  {nombre:25s}: {porcentaje:5.1f}% {barra}\n"

        # Analisis automatico
        total_vegetacion = (
            cobertura["vegetacion_baja"]
            + cobertura["vegetacion_media"]
            + cobertura["vegetacion_alta"]
        )

        reporte += f"\nAnalisis:\n"
        if total_vegetacion > 60:
            reporte += f"  - Area predominantemente verde ({total_vegetacion:.1f}% vegetacion)\n"
        elif cobertura["urbano_o_suelo"] > 40:
            reporte += (
                f"  - Area con alta urbanizacion ({cobertura['urbano_o_suelo']:.1f}%)\n"
            )

        if cobertura["vegetacion_alta"] > 30:
            reporte += f"  - Presencia significativa de vegetacion densa (bosques)\n"

        return reporte


# Crear agente
agente_imagen = ImageAnalysisAgent()
print(f"Agente de analisis creado (dispositivo: {agente_imagen.device})")

In [ ]:
# Ejemplo de uso con datos simulados
import numpy as np

# Simular NDVI para diferentes areas de Pasto
np.random.seed(42)

# Centro urbano: NDVI bajo
ndvi_urbano = np.random.uniform(-0.1, 0.25, (100, 100))

# Zona del Galeras: NDVI mixto (paramo y vegetacion)
ndvi_galeras = np.random.uniform(0.2, 0.7, (100, 100))

# Laguna de la Cocha: NDVI con agua
ndvi_cocha = np.random.uniform(-0.3, 0.5, (100, 100))
ndvi_cocha[40:60, 40:60] = -0.2  # Agua

# Analizar cada area
for nombre, ndvi in [
    ("Centro Urbano de Pasto", ndvi_urbano),
    ("Volcan Galeras", ndvi_galeras),
    ("Laguna de la Cocha", ndvi_cocha),
]:
    cobertura = agente_imagen.analizar_cobertura(ndvi)
    reporte = agente_imagen.generar_reporte(cobertura, nombre)
    print(reporte)

## 6. Agente de Recomendaciones

In [ ]:
class RecommendationAgent:
    """
    Agente que recomienda datos y analisis basado en la tarea.
    """

    def __init__(self):
        self.tareas = {
            "vegetacion": {
                "coleccion": "sentinel-2-l2a",
                "bandas": ["B04", "B08"],
                "indices": ["NDVI", "EVI"],
                "descripcion": "Analisis de salud y cobertura vegetal",
            },
            "agua": {
                "coleccion": "sentinel-2-l2a",
                "bandas": ["B03", "B08", "B11"],
                "indices": ["NDWI", "MNDWI"],
                "descripcion": "Deteccion de cuerpos de agua",
            },
            "urbano": {
                "coleccion": "sentinel-2-l2a",
                "bandas": ["B04", "B08", "B11"],
                "indices": ["NDBI", "UI"],
                "descripcion": "Analisis de areas urbanas y construcciones",
            },
            "elevacion": {
                "coleccion": "cop-dem-glo-30",
                "bandas": ["data"],
                "indices": ["pendiente", "aspecto"],
                "descripcion": "Modelos de elevacion y terreno",
            },
            "uso_suelo": {
                "coleccion": "io-lulc-annual-v02",
                "bandas": ["data"],
                "indices": [],
                "descripcion": "Clasificacion de uso del suelo",
            },
        }

    def recomendar(self, tarea):
        """
        Recomienda datos y metodos para una tarea especifica.
        """
        tarea = tarea.lower()

        if tarea in self.tareas:
            info = self.tareas[tarea]
            return {
                "tarea": tarea,
                "recomendacion": info,
                "mensaje": f"Para '{tarea}' recomiendo usar {info['coleccion']} "
                f"con las bandas {info['bandas']} y calcular {info['indices']}",
            }
        else:
            return {
                "tarea": tarea,
                "mensaje": f"Tarea '{tarea}' no reconocida. Tareas disponibles: {list(self.tareas.keys())}",
            }

    def recomendar_para_pasto(self):
        """
        Genera recomendaciones especificas para San Juan de Pasto.
        """
        recomendaciones = []

        # Basadas en las caracteristicas de Pasto
        recomendaciones.append(
            {
                "area": "Volcan Galeras",
                "tareas": ["elevacion", "vegetacion"],
                "razon": "Monitorear actividad volcanica y paramos",
            }
        )

        recomendaciones.append(
            {
                "area": "Laguna de la Cocha",
                "tareas": ["agua", "vegetacion"],
                "razon": "Monitorear calidad del agua y humedales (Sitio Ramsar)",
            }
        )

        recomendaciones.append(
            {
                "area": "Centro Urbano",
                "tareas": ["urbano", "uso_suelo"],
                "razon": "Analizar expansion urbana y planificacion",
            }
        )

        return recomendaciones


# Crear agente
agente_rec = RecommendationAgent()

# Mostrar recomendaciones para Pasto
print("=== Recomendaciones de Analisis para San Juan de Pasto ===")
for rec in agente_rec.recomendar_para_pasto():
    print(f"\n{rec['area']}:")
    print(f"  Tareas recomendadas: {rec['tareas']}")
    print(f"  Razon: {rec['razon']}")

In [ ]:
# Obtener recomendacion especifica
tarea = "vegetacion"
rec = agente_rec.recomendar(tarea)

print(f"\n=== Recomendacion para tarea: {tarea} ===")
print(rec["mensaje"])
if "recomendacion" in rec:
    print(f"\nDetalles:")
    print(f"  Coleccion: {rec['recomendacion']['coleccion']}")
    print(f"  Bandas: {rec['recomendacion']['bandas']}")
    print(f"  Indices: {rec['recomendacion']['indices']}")

## 7. Pipeline Automatizado

In [ ]:
class GeoAIPipeline:
    """
    Pipeline que combina todos los agentes para analisis automatizado.
    """

    def __init__(self):
        self.agente_busqueda = STACSearchAgent()
        self.agente_imagen = ImageAnalysisAgent()
        self.agente_rec = RecommendationAgent()

    def ejecutar_analisis(self, bbox, tarea, fecha_inicio, fecha_fin):
        """
        Ejecuta un analisis completo automatizado.
        """
        resultados = {}

        # Paso 1: Obtener recomendaciones
        print(f"1. Obteniendo recomendaciones para tarea: {tarea}")
        rec = self.agente_rec.recomendar(tarea)
        resultados["recomendacion"] = rec

        if "recomendacion" not in rec:
            return resultados

        # Paso 2: Buscar datos
        print(f"2. Buscando datos en {rec['recomendacion']['coleccion']}")
        imagenes = self.agente_busqueda.buscar(
            bbox=bbox,
            fecha_inicio=fecha_inicio,
            fecha_fin=fecha_fin,
            coleccion=rec["recomendacion"]["coleccion"],
        )
        resultados["imagenes_encontradas"] = len(imagenes)

        # Paso 3: Seleccionar mejor imagen
        if imagenes:
            mejor = min(imagenes, key=lambda x: x.properties.get("eo:cloud_cover", 100))
            resultados["mejor_imagen"] = {
                "id": mejor.id,
                "fecha": mejor.datetime.strftime("%Y-%m-%d"),
                "nubes": mejor.properties.get("eo:cloud_cover"),
            }
            print(f"3. Mejor imagen: {mejor.datetime.strftime('%Y-%m-%d')}")

        # Paso 4: Generar instrucciones de procesamiento
        print(f"4. Generando instrucciones de procesamiento")
        resultados["instrucciones"] = {
            "bandas_a_descargar": rec["recomendacion"]["bandas"],
            "indices_a_calcular": rec["recomendacion"]["indices"],
            "siguiente_paso": f"Descargar bandas y calcular {rec['recomendacion']['indices']}",
        }

        return resultados


# Crear pipeline
pipeline = GeoAIPipeline()

# Ejecutar analisis para Pasto
print("=== Ejecutando Pipeline de Analisis para Pasto ===")
print("")
resultados = pipeline.ejecutar_analisis(
    bbox=PASTO_BBOX,
    tarea="vegetacion",
    fecha_inicio="2024-01-01",
    fecha_fin="2024-12-31",
)

print("\n=== Resultados del Pipeline ===")
print(f"Imagenes encontradas: {resultados.get('imagenes_encontradas', 0)}")
if "mejor_imagen" in resultados:
    print(f"Mejor imagen: {resultados['mejor_imagen']}")
if "instrucciones" in resultados:
    print(f"\nInstrucciones:")
    print(f"  Bandas: {resultados['instrucciones']['bandas_a_descargar']}")
    print(f"  Indices: {resultados['instrucciones']['indices_a_calcular']}")

## 8. Uso de GeoAI Agents (Avanzado)

GeoAI incluye agentes mas avanzados que pueden interactuar con LLMs.

In [ ]:
# Ejemplo de uso de agentes de GeoAI (requiere API key)
print("GeoAI incluye agentes avanzados que pueden:")
print("")
print("1. GeoAgent: Analisis geoespacial con lenguaje natural")
print("   - 'Encuentra imagenes de Pasto con menos de 10% de nubes'")
print("   - 'Calcula el NDVI promedio para el area urbana'")
print("")
print("2. STACAgent: Busqueda inteligente en catalogos STAC")
print("   - Interpreta consultas en lenguaje natural")
print("   - Recomienda colecciones y parametros")
print("")
print("3. CatalogAgent: Exploracion de datos disponibles")
print("   - Lista y describe colecciones")
print("   - Sugiere datos apropiados para cada tarea")
print("")
print("Para usar estos agentes:")
print("  pip install geoai-py[agents]")
print("  from geoai.agents import GeoAgent")

In [ ]:
# Ejemplo de codigo para usar GeoAgent (requiere API key)
ejemplo_codigo = """
# Instalar dependencias
# pip install geoai-py[agents]

from geoai.agents import GeoAgent
import os

# Configurar API key (OpenAI, Anthropic, etc.)
os.environ["OPENAI_API_KEY"] = "tu-api-key"

# Crear agente
agent = GeoAgent(model="gpt-4")

# Hacer consulta en lenguaje natural
respuesta = agent.chat(
    "Busca imagenes Sentinel-2 de San Juan de Pasto, Colombia "
    "del ultimo mes con menos de 20% de nubes y calcula el NDVI"
)

print(respuesta)
"""

print("Ejemplo de uso de GeoAgent:")
print(ejemplo_codigo)

## 9. Mapa Interactivo de Resultados

In [ ]:
# Crear mapa mostrando las areas de analisis
import geopandas as gpd
from shapely.geometry import box

m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=11, height="600px"
)

m.add_basemap("Esri.WorldImagery")
m.add_basemap("OpenStreetMap")

# Agregar areas de interes
for nombre, bbox in AREAS_INTERES.items():
    geom = box(*bbox)
    gdf = gpd.GeoDataFrame({"nombre": [nombre]}, geometry=[geom], crs="EPSG:4326")
    m.add_gdf(
        gdf,
        layer_name=nombre,
        style={"color": "yellow", "fillOpacity": 0.2, "weight": 2},
    )

# Agregar bbox principal
gdf_bbox = gpd.GeoDataFrame(
    {"nombre": ["Area de Estudio"]}, geometry=[box(*PASTO_BBOX)], crs="EPSG:4326"
)
m.add_gdf(
    gdf_bbox,
    layer_name="Area Total",
    style={"color": "red", "fillOpacity": 0, "weight": 3},
)

m.add_layer_control()
m

## 10. Resumen

En este notebook hemos aprendido a:

1. **Crear agentes de busqueda STAC** para encontrar datos satelitales
2. **Implementar agentes de analisis** que procesan y describen imagenes
3. **Usar agentes de recomendacion** que sugieren datos y metodos
4. **Construir pipelines automatizados** que combinan multiples agentes
5. **Integrar con GeoAI agents** para consultas en lenguaje natural

### Agentes implementados:

- **STACSearchAgent**: Busqueda en catalogos de datos satelitales
- **ImageAnalysisAgent**: Analisis automatico de cobertura del suelo
- **RecommendationAgent**: Recomendaciones de datos y metodos
- **GeoAIPipeline**: Orquestacion de agentes para analisis completo

### Aplicaciones para Pasto:

- Monitoreo del Volcan Galeras
- Analisis de la Laguna de la Cocha
- Seguimiento de expansion urbana
- Evaluacion de cobertura vegetal

### Proximos pasos:

- Integrar con LLMs para consultas en lenguaje natural
- Crear agentes especializados para tareas de Pasto
- Automatizar reportes periodicos